In [1]:
import pyarrow.parquet as pq

In [2]:
df = pq.read_table('../rag/datasets--JetBrains-Research--commit-chronicle/snapshots/5fd076e67b812a9f3d1999e5e40f71715f84bb51/subset_cmg/test-00000-of-00002-983a1050cd3c058c.parquet').to_pandas()

In [3]:
df[(df['language'] == 'JavaScript') & (df['mods'].apply(lambda x: x[0]['diff'].count('.js')==2))].head()['mods'].iloc[0][0]

In [11]:
df['mods'].head()[0][0]['diff'].count('.py')

In [4]:
df.tail()

In [61]:
df['mods'][0][0]['change_type']

In [19]:
for index, row in df.iterrows():
    print(row['message'])
    break

In [5]:
languages = ['Java', 'Python', 'C#', 'C++', 'JavaScript']
language_dict = {language: 0 for language in languages}

In [4]:
import pyarrow.parquet as pq
import glob
import os
from tqdm.auto import tqdm

# Create empty DataFrames for each language
# languages = ['Java', 'Python', 'C++', 'C#', 'JavaScript']
# languages = ['Python', 'JavaScript']
languages = ['Python']
dfs = {lang: [] for lang in languages}

# Create a tqdm progress bar for each language
bars = {lang: tqdm(total=1000, desc=lang) for lang in languages}

folder_path = '../rag/datasets--JetBrains-Research--commit-chronicle/snapshots/5fd076e67b812a9f3d1999e5e40f71715f84bb51/data'  # 文件夹的路径

# files = glob.glob(os.path.join(folder_path, 'train*')) + glob.glob(os.path.join(folder_path, 'validation*'))
files = glob.glob(os.path.join(folder_path, 'test*'))

for file in files:
    df = pq.read_table(file).to_pandas()

    # Iterate over each language
    for lang in languages[:]:
        if lang =='Python':
            lang_suffix = '.py'
        elif lang == 'JavaScript':
            lang_suffix = '.js'

        # Filter rows where language column matches the current language
        lang_df = df[(df['language'] == lang) & (df['mods'].apply(len) == 1) & (df['message'].apply(len) <= 50) & (df['mods'].apply(lambda x: x[0]['change_type']) == 'MODIFY') & (df['mods'].apply(lambda x: len(f"diff --git a/{x[0]['old_path']} b/{x[0]['new_path']} {x[0]['diff']}") <= 500)) & (df['mods'].apply(lambda x: ( (x[0]['old_path'].count(lang_suffix) if x[0]['old_path'] else 0) == 1 and (x[0]['new_path'].count(lang_suffix) if x[0]['new_path'] else 0) == 1 and (x[0]['old_path'].count('.json') if x[0]['old_path'] else 0) == 0 and (x[0]['old_path'].count('.json') if x[0]['old_path'] else 0) == 0 )))]

        # Iterate over each row in the filtered DataFrame
        for index, row in lang_df.iterrows():
            diff = row['mods'][0]
            old_path = 'a/' + diff['old_path']
            new_path = 'b/' + diff['new_path']
            diff_content = diff['diff']  # assume diff_content is an empty string
            item = {
                'msg': row['message'],
                'diff': f"diff --git {old_path} {new_path} {diff_content}",
                'lang': lang
            }
            dfs[lang].append(item)
            bars[lang].update(1)

            # Check if the language has reached 100,000 rows
            if len(dfs[lang]) >= 1000:
                print(f"Reached 1000 rows for {lang}")
                languages.remove(lang)  # Remove language from list to avoid further processing
                break
    # Break out of the loop if all languages have reached 100,000 rows
    if not languages:
        break


Python:   0%|          | 0/1000 [00:00<?, ?it/s]

Reached 1000 rows for Python


In [5]:


import json

# Create a single list of all items
all_items = [item for lang in dfs for item in dfs[lang]]
# Dump the data to a JSON file
with open('../data/final_preprocessed_data/python_baseline_test_data_500.json', 'w') as f:
    json.dump(all_items, f, indent=4)

In [4]:
import json
data = []
while any(value < 1000 for value in language_dict.values()):
    for index, row in df.iterrows():
        language = row['language']
        if language in languages:
            
            if language_dict[language] >= 1000 or len(row['mods']) != 1 or row['mods'][0]['change_type'] != 'MODIFY' or len(f"diff --git a/{row['mods'][0]['old_path']} b/{row['mods'][0]['new_path']} {row['mods'][0]['diff']}") > 3000:
                continue
            language_dict[language] += 1

            item = {
                'msg': row['message'],
                'diff': f"diff --git a/{row['mods'][0]['old_path']} b/{row['mods'][0]['new_path']} {row['mods'][0]['diff']}",
                'lang': language
            }
            data.append(item)

import random
random.seed(42)
random.shuffle(data)

with open('../data/chronicle_data_dev.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, indent=4)


In [5]:
import json

# Create a single list of all items
all_items = [item for lang in dfs for item in dfs[lang]]

import random
random.seed(42)
random.shuffle(all_items)

# Dump the data to a JSON file
with open('../data/chronicle/chronicle_test_data_dev.json', 'w') as f:
    json.dump(all_items, f, indent=4)

In [3]:
len(all_items)

In [66]:
# 遍历每个item
for item in data:
    diff = item['diff']
    new_diff = f"diff --git {diff}"

    item['diff'] = new_diff

# 将更新后的数据写入新的JSON文件
with open('../data/chronicle_data.json', 'w', encoding='UTF-8') as file:
    json.dump(data, file, indent=4)

In [1]:
len(df)

In [1]:
import pyarrow.parquet as pq
import glob
import os
from tqdm.auto import tqdm

# Create empty DataFrames for each language
# languages = ['Java', 'Python', 'C++', 'C#', 'JavaScript']
languages = ['Python', 'JavaScript']
dfs = {lang: [] for lang in languages}

# Create a tqdm progress bar for each language
bars = {lang: tqdm(total=1000, desc=lang) for lang in languages}

folder_path = '../rag/datasets--JetBrains-Research--commit-chronicle/snapshots/5fd076e67b812a9f3d1999e5e40f71715f84bb51/subset_cmg'  # 文件夹的路径

files = glob.glob(os.path.join(folder_path, 'test*'))

for file in files:
    df = pq.read_table(file).to_pandas()

    # Iterate over each language
    for lang in languages[:]:
        # Filter rows where language column matches the current language
        condition = lambda x: all(len(f"diff --git a/{item['old_path']} b/{item['new_path']} {item['diff']}") <= 3000 for item in x)
        type_condition = lambda x: all(item['change_type'] == 'MODIFY' for item in x)
        lang_df = df[(df['language'] == lang) & (df['mods'].apply(len) > 1) & (df['mods'].apply(type_condition)) & (df['mods'].apply(condition))]

        # Iterate over each row in the filtered DataFrame
        for index, row in lang_df.iterrows():
            diffs = {}
            diffs['msg'] = row['message']
            diffs['diffs'] = []
            diffs['lang'] = lang
            for diff in row['mods']:
                old_path = 'a/' + diff['old_path']
                new_path = 'b/' + diff['new_path']
                diff_content = diff['diff']  # assume diff_content is an empty string
                item = {
                    'diff': f"diff --git {old_path} {new_path} {diff_content}",
                }
                diffs['diffs'].append(item)
            merged_diff = '\n'.join(diff['diff'] for diff in diffs['diffs'])
            diffs['diffs'] = merged_diff
            dfs[lang].append(diffs)
            bars[lang].update(1)

            # Check if the language has reached 100,000 rows
            if len(dfs[lang]) >= 1000:
                print(f"Reached 1000 rows for {lang}")
                languages.remove(lang)  # Remove language from list to avoid further processing
                break
    # Break out of the loop if all languages have reached 100,000 rows
    if not languages:
        break

In [3]:
len(dfs['JavaScript'])

In [2]:
import json

# Create a single list of all items
all_items = [item for lang in dfs for item in dfs[lang]]

# Dump the data to a JSON file
with open('../data/chronicle/chronicle_multi_diff_test_data.json', 'w', encoding='UTF-8') as f:
    json.dump(all_items, f, indent=4)